In [1]:
import functools
import jax
import os
import html

from datetime import datetime
from jax import numpy as jp
jax.config.update("jax_debug_nans", False)
jax.config.update("jax_enable_x64", True)
import matplotlib.pyplot as plt

from IPython.display import HTML, clear_output, display
from brax.io import html

import shutil

from src.pods.Pods import train as train_pods
from src.hds.Hds import train as train_hds
from src.envs.original import Pendulum
from src.dyn_model.Predict import pretrained_params




In [2]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)
print(jax.devices())

cpu
[CpuDevice(id=0)]


In [3]:
env = Pendulum.Pendulum()
state = jax.jit(env.reset)(rng=jax.random.PRNGKey(seed=0))

#HTML(html.render(env.sys, [state.pipeline_state]))

11


In [4]:
ckpt_dir = './tmp/flax_ckpt'

if os.path.exists(ckpt_dir):
    shutil.rmtree(ckpt_dir)  # Remove any existing checkpoints from the last notebook run.

In [5]:

def progress_f(x_data,y_data,epoch,reward):
    print(reward)
    x_data.append(epoch)
    y_data.append(reward)
    clear_output(wait=True)
    plt.xlabel('epoch')
    plt.ylabel('total reward')
    plt.plot(x_data, y_data)
    plt.show()

policy = train_hds(env,trajectory_length=100 ,num_samples=100,epochs=20, inner_epochs=20, alpha_a=1.0, aggregation_factor_beta=1.0,init_learning_rate=1e-3, init_noise=0.1, noise_decay=0.5, progress_fn=progress_f) 
# policy = train_pods(env,trajectory_length=100 ,num_samples=50,epochs=20, inner_epochs=20, alpha_a=8e-5, init_learning_rate=1e-3, progress_fn=progress_f)

11


In [1]:
bestest_env = Pendulum.Pendulum()

def unroll_policy_good_env(policy, env, num_steps=1000):
    state = env.reset(rng=jax.random.PRNGKey(seed=0))
    states = [state]
    actions = []
    rewards = []
    for _ in range(num_steps):
        action = policy(jp.array(state.obs)).mode()
        state = env.step(state, action)
        states.append(state)
        actions.append(action)
        rewards.append(state.reward)
    return states, actions, rewards

states, actions, rewards = unroll_policy_good_env(policy, bestest_env, num_steps=1000)


NameError: name 'Pendulum' is not defined

In [ ]:

yes=html.render(env.sys.replace(dt=env.dt), rollout)
with open("data.html", "w") as file:
    file.write(yes)